# ¿Qué es una API?

## Definición  
Una **API** (*Application Programming Interface* o *Interfaz de Programación de Aplicaciones*) es un conjunto de reglas y definiciones que permite que diferentes aplicaciones se comuniquen entre sí. Actúa como un intermediario que facilita la interacción entre software sin necesidad de que los desarrolladores comprendan los detalles internos de su implementación.

## ¿Cómo funciona una API?  
Una API funciona como un puente que permite que una aplicación solicite información o servicios a otra aplicación. Este proceso generalmente sigue estos pasos:

1. **Solicitud**: Un cliente (como una aplicación web o móvil) envía una petición a la API con ciertos parámetros.  
2. **Procesamiento**: La API interpreta la solicitud, consulta la base de datos o realiza una acción específica.  
3. **Respuesta**: La API devuelve la información solicitada en un formato estructurado, generalmente **JSON** o **XML**.

## Tipos de API  
Las API pueden clasificarse en diferentes categorías según su propósito y accesibilidad:

- **APIs abiertas (públicas)**: Disponibles para cualquier desarrollador sin restricciones. Ejemplo: API de OpenWeather.  
- **APIs privadas**: Usadas internamente dentro de una organización.  
- **APIs de socios**: Disponibles solo para socios comerciales autorizados.  
- **APIs compuestas**: Combinan múltiples API para ofrecer funcionalidades avanzadas.  

## Ejemplo de una API REST  
Una API RESTful es un tipo de API que sigue principios como el uso de recursos y métodos HTTP. Un ejemplo de solicitud para obtener información de un usuario en una API REST podría ser:

```http
GET https://api.ejemplo.com/categoria/fecha
```

En este caso los parámetros serían la URL, la categoria y la fecha

# Tabla de Métodos HTTP

| Método  | Descripción | Uso Principal | Ejemplo de Endpoint |
|---------|------------|--------------|---------------------|
| **GET** | Obtiene datos del servidor sin modificar nada. | Leer información | `GET /usuarios/123` |
| **POST** | Crea un nuevo recurso en el servidor. | Crear datos | `POST /usuarios` |
| **PUT** | Actualiza completamente un recurso existente. | Reemplazar datos | `PUT /usuarios/123` |
| **PATCH** | Modifica parcialmente un recurso existente. | Actualizar parcialmente | `PATCH /usuarios/123` |
| **DELETE** | Elimina un recurso del servidor. | Borrar datos | `DELETE /usuarios/123` |



### Ejemplo de interacción de sistemas a través de API

**GET**: Un cliente navega por la web de compra de instrumentos y filtra por "guitarras".
```http
GET https://api.tienda.com/productos/guitarras ```

**POST**: Selecciona una que le gusta y la añade al carrito. Se produce un post en la base de datos, ya que se crea un registro nuevo, el clienteX añade un producto a su carrito.

```http
POST  https://api.tienda.com/usuarioX/carrito/productoG ```

## RESTful API

Las APIs que nosotros vamos a usar son RESTful.

El término RESTful se refiere a sistemas o APIs que siguen los principios de REST (Representational State Transfer), un estilo de arquitectura para servicios web.

Cuando una API es RESTful, significa que:
 - Usa los métodos HTTP estándar (GET, POST, PUT, DELETE).
 - Opera con recursos representados por URLs (/usuarios, /productos/123).
 - Devuelve datos en formatos estándar, generalmente JSON o XML.
 - Es stateless (sin estado), es decir, cada solicitud contiene toda la información necesaria sin depender de sesiones en el servidor.
 
Si una API no es RESTful, puede pertenecer a otros tipos de arquitectura, como **SOAP, GraphQL, RPC o WebSockets**. Pero estas no las vamos a trabajar

## Extracción de datos de una API

Desde el punto de vista del científico de datos una API es una base de datos de la que recolectar la información para posteriormente hacer sus análisis.

Vamos a ver cómo extraer esos datos con Python en una API de la Red Eléctrica Española, con información de la producción y demanda energética en España  



In [87]:
import pandas as pd
import numpy as np
import os
import requests
import datetime
import matplotlib.pyplot as plt
from matplotlib import ticker

import seaborn as sns
sns.set()


import warnings
warnings.filterwarnings("ignore")

Entrando en la [documentación](https://www.ree.es/es/datos/apidatos), tenemos algunas pistas y ayudas de cómo configurar nuestras llamadas a la API.

En **URI definition** podemos elegir el lenguaje, la categoría (mercado, balance, demanda, generación..) y el wiget (percio, evolucion...).

También podemos definir parámetros en función de la region o localizacion

En **Example request** tenemos un ejemplo que podemos usar para empezar a probar.

Usando el método `requests.get` y hacemos llamada a la API. Con el método `.json()` transofrmamos la respuesta a JSON. Que es principalmente una manera de almacenar información similar a un diccionario, con `keys()` y `values()`

In [88]:
url_balance = 'https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2019-01-01T00:00&end_date=2019-01-31T23:59&time_trunc=day'

res_balance = requests.get(url = url_balance)
balance = res_balance.json()

In [89]:
balance

{'data': {'type': 'Balance de energía eléctrica',
  'id': 'bal1',
  'attributes': {'title': 'Balance de energía eléctrica',
   'last-update': '2020-01-10T08:15:41.000+01:00',
   'description': 'Balance eléctrico: asignación de unidades de producción según combustible principal.'},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2025-04-06T06:21:43'}}},
 'included': [{'type': 'Renovable',
   'id': 'Renovable',
   'attributes': {'title': 'Renovable',
    'last-update': '2020-01-10T08:15:21.000+01:00',
    'description': None,
    'magnitude': None,
    'content': [{'type': 'Hidráulica',
      'id': '10288',
      'groupId': 'Renovable',
      'attributes': {'title': 'Hidráulica',
       'description': '10288',
       'color': '#0090d1',
       'icon': None,
       'type': 'distinct',
       'magnitude': None,
       'composite': False,
       'last-update': '2020-01-10T08:15:21.000+01:00',
       'values': [{'value': 53722.12,
         'percentage': 0.31722207661963636,
   

In [93]:
def esquema(dic, nivel=0):
    indent = "  " * nivel  # Espacios para visualización
    if isinstance(dic, dict):
        for key, value in dic.items():
            print(f"{indent}- {key}: {type(value).__name__}")  # Muestra clave y tipo de valor
            esquema(value, nivel + 1)  # Llamada recursiva si hay diccionarios anidados
    elif isinstance(dic, list):
        print(f"{indent}- Lista ({len(dic)} elementos)")
        if dic:  # Si la lista no está vacía, analiza el primer elemento
            esquema(dic[0], nivel + 1)


esquema(balance)


- data: dict
  - type: str
  - id: str
  - attributes: dict
    - title: str
    - last-update: str
    - description: str
  - meta: dict
    - cache-control: dict
      - cache: str
      - expireAt: str
- included: list
  - Lista (4 elementos)
    - type: str
    - id: str
    - attributes: dict
      - title: str
      - last-update: str
      - description: NoneType
      - magnitude: NoneType
      - content: list
        - Lista (8 elementos)
          - type: str
          - id: str
          - groupId: str
          - attributes: dict
            - title: str
            - description: str
            - color: str
            - icon: NoneType
            - type: str
            - magnitude: NoneType
            - composite: bool
            - last-update: str
            - values: list
              - Lista (31 elementos)
                - value: float
                - percentage: float
                - datetime: str
            - total: float
            - total-percentage: 

Vamos a navegar por el json y ver qué información nos proporciona 

In [22]:
balance.keys()

dict_keys(['data', 'included'])

In [23]:
balance['included']

[{'type': 'Renovable',
  'id': 'Renovable',
  'attributes': {'title': 'Renovable',
   'last-update': '2020-01-10T08:15:21.000+01:00',
   'description': None,
   'magnitude': None,
   'content': [{'type': 'Hidráulica',
     'id': '10288',
     'groupId': 'Renovable',
     'attributes': {'title': 'Hidráulica',
      'description': '10288',
      'color': '#0090d1',
      'type': 'distinct',
      'magnitude': None,
      'composite': False,
      'last-update': '2020-01-10T08:15:21.000+01:00',
      'values': [{'value': 53722.12,
        'percentage': 0.31722207661963636,
        'datetime': '2019-01-01T00:00:00.000+01:00'},
       {'value': 61294.768,
        'percentage': 0.20827477009671783,
        'datetime': '2019-01-02T00:00:00.000+01:00'},
       {'value': 82981.951,
        'percentage': 0.3622758196846154,
        'datetime': '2019-01-03T00:00:00.000+01:00'},
       {'value': 94301.94,
        'percentage': 0.4482532404733024,
        'datetime': '2019-01-04T00:00:00.000+01:00'

In [29]:
print("tipo de objeto ", type(balance['included']))
print("longitud ", len(balance['included']))

tipo de objeto  <class 'list'>
longitud  4


In [33]:
for i in range(len(balance['included'])):
    print("tipo", type(balance['included'][i]), "nombre", balance['included'][i]['type'])

tipo <class 'dict'> nombre Renovable
tipo <class 'dict'> nombre No-Renovable
tipo <class 'dict'> nombre Almacenamiento
tipo <class 'dict'> nombre Demanda


En conclusión, Qué hemos visto que contiene?

balance es un diccionario que contiene dentro una serie de diccionarios y listas y más diccionarios en su interior. En un primer nivel tenemos dos diccionarios 'data' y 'included'. En data vemos una serie de meta-datos como la descripcion del documento.

En included es donde tenemos el grosso de la información. Contiene una lista de 4 elementos (diccionarios). 

En definitiva hay 2 diccionarios con la producción energética por días, fuente de generación y tipo de energía (Renovable o no-renovable), almacenados en diccionarios más profundos. También contamos con un diccionario con el almacenamiento y otro con la demanda.

Por ejemplo el 1 de enero de 2019 se generó 53722 Kw (?) de fuente hidráulica y representaron el 31% de la producción del día

In [40]:
balance['included'][0]['attributes']['content'][0]['attributes']['values'][0]

{'value': 53722.12,
 'percentage': 0.31722207661963636,
 'datetime': '2019-01-01T00:00:00.000+01:00'}

## Ejercicio intermedio

Navega por el diccionario y obtén una lista con los nombres de todas las fuentes renovables y otra con las no renovables.

In [48]:
balance['included'][0]['attributes']['content']

[{'type': 'Hidráulica',
  'id': '10288',
  'groupId': 'Renovable',
  'attributes': {'title': 'Hidráulica',
   'description': '10288',
   'color': '#0090d1',
   'type': 'distinct',
   'magnitude': None,
   'composite': False,
   'last-update': '2020-01-10T08:15:21.000+01:00',
   'values': [{'value': 53722.12,
     'percentage': 0.31722207661963636,
     'datetime': '2019-01-01T00:00:00.000+01:00'},
    {'value': 61294.768,
     'percentage': 0.20827477009671783,
     'datetime': '2019-01-02T00:00:00.000+01:00'},
    {'value': 82981.951,
     'percentage': 0.3622758196846154,
     'datetime': '2019-01-03T00:00:00.000+01:00'},
    {'value': 94301.94,
     'percentage': 0.4482532404733024,
     'datetime': '2019-01-04T00:00:00.000+01:00'},
    {'value': 58822.692,
     'percentage': 0.27465081477001124,
     'datetime': '2019-01-05T00:00:00.000+01:00'},
    {'value': 42064.747,
     'percentage': 0.17106178088210705,
     'datetime': '2019-01-06T00:00:00.000+01:00'},
    {'value': 66708.37

## Parámetros de la llamada API REE



#### ejemplos de las diferentes llamadas que podemos hacer

In [76]:
url_demanda = 'https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?start_date=2018-01-01T00:00&end_date=2018-01-01T23:59&time_trunc=hour'
url_generacion  = 'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2018-12-30T00:00&end_date=2018-12-31T23:59&time_trunc=hour'
url_intercambios = 'https://apidatos.ree.es/es/datos/intercambios/portugal-frontera?start_date=2018-12-31T00:00&end_date=2018-12-31T23:59&time_trunc=day'
url_transporte = 'https://apidatos.ree.es/es/datos/transporte/energia-no-suministrada-ens?start_date=2022-12-31T00:00&end_date=2022-12-31T23:59&time_trunc=month'
url_balance = 'https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2018-01-01T00:00&end_date=2018-01-01T23:59&time_trunc=day'
# url_demanda = 'https://apidatos.ree.es/en/datos/demanda/ire-general?start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=month'
url_balance = 'https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2019-01-01T00:00&end_date=2019-01-31T23:59&time_trunc=day'
url_precio  = 'https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=hour'
url_precio  = 'https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=hour'


De cara a una futura **automatizacion de la llamada** o a una serie de llamadas recurrentes vamos a definir los parámetros que van a definir qué datos vamos a extraer.

In [49]:
lang = 'es'
start_date = '2024-01-01T00:00'
end_date = '2024-01-05T23:59'
time_trunc = 'day' # 'month' 'day' 'hour'
categoria = 'balance' # 'generacion','intercambios',' transporte','balance','mercados','demanda'
widget = 'balance-electrico'

In [50]:
url_balance = f'https://apidatos.ree.es/{lang}/datos/{categoria}/{widget}?start_date={start_date}&end_date={end_date}&time_trunc={time_trunc}'
print(url_balance)

https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2024-01-01T00:00&end_date=2024-01-05T23:59&time_trunc=day


#### Efectuamos la llamada

Para ello usamos el método `requests.get()` e intriducimos la url. Que desde cierto punto de vista es una query a una base de datos.

In [77]:
res_demanda = requests.get(url = url_demanda)
demanda = res_demanda.json()

# res_generacion = requests.get(url = url_generacion)
# generacion = res_generacion.json()
# generacion

# res_intercambio = requests.get(url = url_intercambios)
# intercambio = res_intercambio.json()
# intercambio

# res_transporte = requests.get(url = url_transporte)
# transporte = res_transporte.json()
# transporte

# res_precio = requests.get(url = url_precio)
# precio = res_precio.json()

# res_balance = requests.get(url = url_balance)
# balance = res_balance.json()

In [52]:
balance['included'][0]['attributes']['content'][0]['attributes']['values']

[{'value': 58381.148,
  'percentage': 0.19718853720380722,
  'datetime': '2024-01-01T00:00:00.000+01:00'},
 {'value': 68004.167,
  'percentage': 0.15570440748011716,
  'datetime': '2024-01-02T00:00:00.000+01:00'},
 {'value': 93862.43,
  'percentage': 0.22015731350369466,
  'datetime': '2024-01-03T00:00:00.000+01:00'},
 {'value': 139605.013,
  'percentage': 0.44288074417231343,
  'datetime': '2024-01-04T00:00:00.000+01:00'},
 {'value': 83467.494,
  'percentage': 0.17956310101675974,
  'datetime': '2024-01-05T00:00:00.000+01:00'}]

## Transformación de las llamadas a la API en una tabla de datos manejable

Ya conocemos perfectamente los datos de que disponemos, lo que nos interesa y cómo extraerlo. Ahora vamos a almacenar toda esa información en un dataframe.

Para ello vamos a navegar nuevamente a través del json balance. También vamos a obtener la información de la demanda, para enriquecer nuestro df

### Repetimos la llamada para tenerla a mano

In [53]:
lang = 'es'
start_date = '2024-01-01T00:00'
end_date = '2024-01-05T23:59'
time_trunc = 'day' # 'month' 'day' 'hour'
categoria = 'balance' # 'generacion','intercambios',' transporte','balance','mercados','demanda'
widget = 'balance-electrico'

url_balance = f'https://apidatos.ree.es/{lang}/datos/{categoria}/{widget}?start_date={start_date}&end_date={end_date}&time_trunc={time_trunc}'
res_balance = requests.get(url = url_balance)
balance = res_balance.json()

In [54]:
energias=[]
suma = []
demanda = []
precios = []

for i in range(len(balance['included'][0]['attributes']['content'][0]['attributes']['values'])):
    print(balance['included'][0]['attributes']['content'][0]['attributes']['values'][i]['datetime'])
    
    data = balance['included'][0]['attributes']['content'][0]['attributes']['values'][i]['datetime']
    hidra = balance['included'][0]['attributes']['content'][0]['attributes']['values'][i]['value']
    eolica = balance['included'][0]['attributes']['content'][1]['attributes']['values'][i]['value']
    solar_foto = balance['included'][0]['attributes']['content'][2]['attributes']['values'][i]['value']
    solar_term = balance['included'][0]['attributes']['content'][3]['attributes']['values'][i]['value']
    hidro_eol = balance['included'][0]['attributes']['content'][4]['attributes']['values'][i]['value']
    otras_ren = balance['included'][0]['attributes']['content'][5]['attributes']['values'][i]['value']
    resid_ren = balance['included'][0]['attributes']['content'][6]['attributes']['values'][i]['value']
    total_ren = balance['included'][0]['attributes']['content'][7]['attributes']['values'][i]['value']
    
#     turb_bomb = balance['included'][1]['attributes']['content'][0]['attributes']['values'][i]['value']
    nuclear = balance['included'][1]['attributes']['content'][0]['attributes']['values'][i]['value']
    cicl_comb = balance['included'][1]['attributes']['content'][1]['attributes']['values'][i]['value']
    carbon = balance['included'][1]['attributes']['content'][2]['attributes']['values'][i]['value']
    motor_dies = balance['included'][1]['attributes']['content'][3]['attributes']['values'][i]['value']
    turb_gas = balance['included'][1]['attributes']['content'][4]['attributes']['values'][i]['value']
    turb_vapor = balance['included'][1]['attributes']['content'][5]['attributes']['values'][i]['value']
    cogeneraci = balance['included'][1]['attributes']['content'][6]['attributes']['values'][i]['value']
    resid_Nren = balance['included'][1]['attributes']['content'][7]['attributes']['values'][i]['value']
    total_Nren = balance['included'][1]['attributes']['content'][8]['attributes']['values'][i]['value']
    
    demanda = balance['included'][2]['attributes']['content'][2]['attributes']['values'][i]['value']
        
    energias.append([data,hidra,eolica,solar_foto, solar_term, hidro_eol, otras_ren, resid_ren,total_ren,
                    nuclear,cicl_comb,carbon,motor_dies,turb_gas,turb_vapor,cogeneraci,resid_Nren,total_Nren,
                    demanda]) # turb_bomb,
    
energias

2024-01-01T00:00:00.000+01:00
2024-01-02T00:00:00.000+01:00
2024-01-03T00:00:00.000+01:00
2024-01-04T00:00:00.000+01:00
2024-01-05T00:00:00.000+01:00


[['2024-01-01T00:00:00.000+01:00',
  58381.148,
  159629.924,
  66642.505,
  2768.958,
  3.65,
  6171.128,
  2470.343,
  296067.656,
  171096.199,
  84443.792,
  6327.863,
  6206.73,
  747.236,
  2705.07,
  29068.821,
  3332.929,
  303928.63999999996,
  19.879],
 ['2024-01-02T00:00:00.000+01:00',
  68004.167,
  307120.989,
  49615.015,
  2760.043,
  2.92,
  6820.63,
  2427.951,
  436751.715,
  171019.317,
  68112.492,
  6972.359,
  7293.833,
  1155.047,
  3404.82,
  38357.135,
  3282.372,
  299597.375,
  24.866],
 ['2024-01-03T00:00:00.000+01:00',
  93862.43,
  284990.434,
  36440.805,
  1063.676,
  3.037,
  7570.068,
  2412.098,
  426342.548,
  170924.516,
  74598.543,
  8052.608,
  7361.897,
  1241.935,
  3344.13,
  46540.326,
  3125.789,
  315189.744,
  19.59],
 ['2024-01-04T00:00:00.000+01:00',
  139605.013,
  138827.131,
  26019.065,
  1.363,
  9.976,
  8209.237,
  2548.542,
  315220.32700000005,
  171083.241,
  92808.246,
  7966.044,
  6899.995,
  1343.638,
  3229.634,
  58299.82

In [75]:
url_precio  = 'https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real'
precios=requests.get(url_precio)

precios

<Response [500]>

In [26]:
suma = []
precios = []
for j in range(len(precio['included'][0]['attributes']['values'])):
    suma.append(precio['included'][0]['attributes']['values'][j]['value'])
    if (j+1) % 24 == 0:
        precios.append([precio['included'][0]['attributes']['values'][j]['datetime'],np.mean(suma),np.std(suma)])
        suma = []
precios = pd.DataFrame(precios, columns = ['date','precio_avg','precio_std'])

NameError: name 'precio' is not defined

In [242]:
precios['date'] = pd.to_datetime(precios['date'].str[:10],format = '%Y-%m-%d')
precios

,date,precio_avg,precio_std
0,2018-12-30,58.49375,7.320352
1,2018-12-31,61.79500,6.069259


In [56]:
energ = pd.DataFrame(energias, columns = ['date','hidra','eolica','solar_foto', 'solar_term', 'hidro_eol', 'otras_ren', 'resid_ren',
                                  'total_ren','nuclear','cicl_comb','carbon','motor_dies','turb_gas',
                                  'turb_vapor','cogeneraci','resid_Nren','total_Nren','demanda'])
energ['date'] = pd.to_datetime(energ['date'].str[:10],format = '%Y-%m-%d')
energ

,date,hidra,eolica,solar_foto,solar_term,hidro_eol,otras_ren,resid_ren,total_ren,nuclear,cicl_comb,carbon,motor_dies,turb_gas,turb_vapor,cogeneraci,resid_Nren,total_Nren,demanda
0,2024-01-01,58381.148,159629.924,66642.505,2768.958,3.650,6171.128,2470.343,296067.656,171096.199,84443.792,6327.863,6206.730,747.236,2705.070,29068.821,3332.929,303928.640,19.879
1,2024-01-02,68004.167,307120.989,49615.015,2760.043,2.920,6820.630,2427.951,436751.715,171019.317,68112.492,6972.359,7293.833,1155.047,3404.820,38357.135,3282.372,299597.375,24.866
2,2024-01-03,93862.430,284990.434,36440.805,1063.676,3.037,7570.068,2412.098,426342.548,170924.516,74598.543,8052.608,7361.897,1241.935,3344.130,46540.326,3125.789,315189.744,19.590
3,2024-01-04,139605.013,138827.131,26019.065,1.363,9.976,8209.237,2548.542,315220.327,171083.241,92808.246,7966.044,6899.995,1343.638,3229.634,58299.828,3924.826,345555.452,19.925
4,2024-01-05,83467.494,306634.568,61906.689,2258.275,7.722,8140.782,2421.029,464836.559,171090.338,64429.348,7141.985,7125.165,1365.288,2899.190,53839.946,3753.182,311644.442,20.784


In [254]:
REE_df = energ.merge(precios, on = 'date', how = 'left').round(2)
REE_df

,date,hidra,eolica,solar_foto,solar_term,hidro_eol,otras_ren,resid_ren,total_ren,turb_bomb,...,carbon,motor_dies,turb_gas,turb_vapor,cogeneraci,resid_Nren,total_Nren,demanda,precio_avg,precio_std
0,2018-12-30,67724.77,120204.23,18328.50,7300.84,14.49,8419.94,2692.41,224685.18,1856.26,...,41887.77,8249.04,1196.81,7350.23,77956.68,6378.35,356536.39,638506.18,58.49,7.32
1,2018-12-31,73308.02,76388.13,19115.74,7941.76,17.98,8725.23,2645.48,188142.34,484.09,...,57940.10,8050.00,1339.02,8225.95,69140.16,5970.05,394646.03,655175.24,61.79,6.07


In [94]:
url_ue='https://data.europa.eu/api/hub/search/resources'
ue=requests.get(url_ue)
ue

<Response [200]>

In [95]:
ue.json()

['service-offering', 'editorial-content']

In [106]:
url_ue='https://data.europa.eu/api/hub/search/catalogues/data-gov-uk'
ue=requests.get(url_ue)
ue

<Response [200]>

In [107]:
ue.json()

{'result': {'country': {'resource': 'http://publications.europa.eu/resource/authority/country/GBR',
   'label': 'United Kingdom',
   'id': 'gb'},
  'creator': None,
  'catalog': None,
  'is_part_of': None,
  'description': {'en': 'Open Data Portal United Kingdom'},
  'language': [{'resource': 'http://publications.europa.eu/resource/authority/language/ENG',
    'id': 'en',
    'label': 'English'}],
  'title': {'en': 'data.gov.uk'},
  'theme_taxonomy': None,
  'license': None,
  'spatial_resource': None,
  'rights': None,
  'publisher': {'name': 'Government Digital Service',
   'type': 'Agent',
   'email': 'mailto:info@digital.cabinet-office.gov.uk',
   'homepage': 'https://www.gov.uk/government/organisations/government-digital-service'},
  'modified': '2023-02-10T04:05:59.249901Z',
  'id': 'data-gov-uk',
  'has_part': None,
  'spatial': [],
  'issued': '2022-11-21T05:36:42Z',
  'homepage': 'https://data.gov.uk',
  'count': 54677}}

In [71]:
import requests

# Definir la URL de autenticación
url = "https://data.europa.eu/auth/middleware/login/service"

# Definir los datos de autenticación
payload = {
    "client_id": "bvazquezlin@gmail.com",
    "client_secret": "Dataeurope6!"
}

# Definir los encabezados
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Enviar la solicitud POST
response = requests.post(url, json=payload, headers=headers)

# Verificar la respuesta
if response.status_code == 200:
    data = response.json()
    access_token = data.get("access_token")
    print("Token obtenido:", access_token)
else:
    print("Error en la autenticación:", response.status_code, response.text)


Error en la autenticación: 401 {
  "error" : "authentication_failed",
  "error_description" : "Invalid client or Invalid client credentials"
}
